In [123]:
# EX2: Train the GPT on your own dataset of choice! What other data could be fun to blabber on about?
#      A fun advanced suggestion if you like: train a GPT to do addition of two numbers, i.e. a+b=c. 
#      You may find it helpful to predict the digits of c in reverse order, as the typical addition 
#       algorithm (that you're hoping it learns) would proceed right to left too. 
#      You may want to modify the data loader to simply serve random problems and skip the generation 
#       of train.bin, val.bin. 
#      You may want to mask out the loss at the input positions of a+b that just specify the problem 
#       using y=-1 in the targets (see CrossEntropyLoss ignore_index). 
#      Does your Transformer learn to add? 
#      Once you have this, swole doge project: build a calculator clone in GPT, for all of +-*/. 
#      Not an easy problem. You may need Chain of Thought traces.

In [124]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [125]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5001
eval_interval = 500
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eval_iters = 200
n_embd = 32
n_head = 4 
n_layer = 4
dropout = 0.1

In [126]:
# prepare data

text = ""

for _ in range(10000):
    op_id = torch.multinomial(torch.ones(4), num_samples=1).item()
    operation = ["+", "-", "*", "/"][op_id]
    operation_token = ["add", "sub", "mul", "div"][op_id]
    num1 = torch.randint(0, 100, (1,)).item()
    num2 = torch.randint(0, 100, (1,)).item()

    text += f"{num1} {operation} {num2} = " # question
    text += f"<{operation_token}> {num1} {num2} <end> <Stop> " # solution

len(text)


In [128]:
chars = sorted(list(set(text.split())))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[w] for w in s.split()]
decode = lambda l: ' '.join([itos[i] for i in l])
vocab_size

111

In [138]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
len(train_data), len(val_data)

(81000, 9000)

In [130]:
def get_batch(split):
    # generate a small batch of data inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [131]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [132]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.Hn = num_heads
        self.Hs = head_size
        self.attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # key, query and value as one batch
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        Hs = self.Hs
        Hn = self.Hn
        B, T, C = x.shape
        q, k, v = self.attn(x.float()).split(n_embd, dim=-1)
        q = q.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        k = k.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        v = v.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * Hs**-0.5 # (B, Hn, T, Hs) @ (B, Hn, Hs, T) ---> (B, Hn, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, Hn, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        out = wei @ v # (B, Hn, T, T) @ (B, Hn, T, Hs) ---> (B, Hn, T, Hs)

        out = out.transpose(1, 2).contiguous().view(B, T, C) # (B, Hn, T, Hs) ---> (B, T, Hn * Hs)
        out = self.proj(out) # from MHA
        out = self.dropout(out) # from MHA
        return out

class FeedForward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd lile
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class GPTModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd=n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens=500):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            if idx_next == stoi['<Stop>']:
                return idx
        return idx

In [133]:
model = GPTModel()
m = model.to(device)

print(f"Parameter count: {sum(p.numel() for p in m.parameters())/1e6} M parameters")

# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Parameter count: 0.058735 M parameters


In [134]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.8771, val loss 4.8798
step 500: train loss 2.2373, val loss 2.2397
step 1000: train loss 2.2247, val loss 2.2311
step 1500: train loss 1.4230, val loss 1.4280
step 2000: train loss 1.3283, val loss 1.3292
step 2500: train loss 1.3208, val loss 1.3256
step 3000: train loss 1.3198, val loss 1.3240
step 3500: train loss 1.3187, val loss 1.3207
step 4000: train loss 1.3183, val loss 1.3215
step 4500: train loss 1.3171, val loss 1.3221
step 5000: train loss 1.3167, val loss 1.3231


In [135]:
def query(q):
    def resolve(answer):

        end_token = '<end>'
        for op_token in ['<add>', '<sub>', '<mul>', '<div>']:
            if op_token in answer:
                operation = op_token
                break
        resolvable_substring = answer[answer.find(op_token):answer.find(end_token)+len(end_token)]

        numbers = answer[answer.find(op_token)+len(op_token):answer.find(end_token)].split()
        
        result = None
        if operation == '<add>': result = sum(int(n) for n in numbers)
        elif operation == '<sub>': result = int(numbers[0]) - sum(int(n) for n in numbers[1:])
        elif operation == '<mul>': result = torch.prod(torch.tensor([int(n) for n in numbers])).item()
        elif operation == '<div>': result = int(numbers[0]) / torch.prod(torch.tensor([int(n) for n in numbers[1:]])).item()

        answer = answer.replace(resolvable_substring, str(result))
        return answer.replace('<Stop>', '')

    context = torch.tensor(encode(q))
    prediction = decode(m.generate(context.view(1, -1))[0].tolist())
    print(resolve(prediction))

In [136]:
query("10 + 20 = ")
query("30 - 40 = ")
query("50 * 60 = ")
query("70 / 80 = ")

10 + 20 = 30 
30 - 40 = -10 
50 * 60 = 3000 
70 / 80 = 0.875 


In [ ]:
# I knew it's a bit hacky, but it works and needs less training :)